Here's how to use the `scoby` code! `scoby` (`S`pectra from `C`atalogs of `OB` stars) can help you make synthetic spectra and related products from real observed stellar catalogs. The spectra are based on the [PoWR](https://www.astro.physik.uni-potsdam.de/~wrh/PoWR/) models.

This example will walk through making a G<sub>0</sub> map from an example catalog



# Part 1: Converting a text catalog to a CatalogResolver object
The first step is to create a CatalogResolver object using your stellar catalog table.

The absolute first step is to import the package. You might have to alter this path depending on where you put this notebook. The path should point to the main repository directory.

In [ ]:
import sys
# Can set up a temporary path to wherever the scoby code is 
sys.path.append('../scoby')

import scoby
print(scoby.__version__)

Load up a catalog as a DataFrame. Getting your catalog to this point is not trivial, and it is often a very case-specific process so I couldn't write much generalizable code to help with this. There are some potentially helpful example functions in `scoby.parsing_utils`.

This notebook uses a catalog that is more or less ready to go. These first few cells will show a little bit of catalog processing and reduction using `pandas`.

In [ ]:
import os
import scoby.config
# This creates a Pandas DataFrame, which is a convenient way to work with catalogs
# The test cluster is Westerlund 2, in the region RCW 49
df = scoby.config.load_test_data()

# Make some adjustments to the DataFrame
del df['MSP']
df.rename(columns={'Unnamed: 0': 'ID'}, inplace=True)
df.set_index("ID", inplace=True)
df = df[df['SpectralType']!='ET']

# Show the table
df

In [ ]:
# Convert the coordinates to SkyCoord objects
from astropy.coordinates import SkyCoord

df['SkyCoord'] = df.filter(['RAdeg', 'DEdeg']).apply(lambda row : SkyCoord(row.RAdeg, row.DEdeg, unit='deg'), axis=1)
df

In [ ]:
# Filter out faraway stars
from astropy import units as u

center_coord = SkyCoord("10 23 58.1 -57 45 49", unit=(u.hourangle, u.deg), frame='fk5') # Specific to this cluster
def within_5_arcmin(coord):
    return coord.separation(center_coord) < 5*u.arcmin

df = df[df['SkyCoord'].apply(within_5_arcmin)]
df

Now we can convert to a `CatalogResolver` object, which `scoby` provides to represent a catalog of stars.

In [ ]:
# First load up the 3 sets of tables that make the CatalogResolver function: Leitherer, Martins, and PoWR
# Load up PoWR models
powr_tables = {x: scoby.spectral.powr.PoWRGrid(x) for x in ('OB', 'WNL-H50', 'WNL')}
# Load up Martins+2005 calibration tables
calibration_tables = scoby.spectral.sttable.STTable(*scoby.spectral.martins.load_tables_df())
# Load up Leitherer tables
ltables = scoby.spectral.leitherer.LeithererTable()

# Create CatalogReslover and initialize with these tables
# This uses the "SpectralType" column in this case -- all it needs are a list of string spectral types
catr = scoby.spectral.stresolver.CatalogResolver(df['SpectralType'].values,
                                                 calibration_table=calibration_tables,
                                                 leitherer_table=ltables,
                                                 powr_dict=powr_tables)
# At this point, every star has been connected to a PoWR model

In [ ]:
print(catr)

In [ ]:
for star in catr:
    print(star)

## 1.1: Obtain basic cluster properties
Once we have made the CatalogResolver, we can start querying cluster properties.
Single-value quantities (like total mass, total FUV luminosity, etc) are the easiest to get.
By default, `scoby` handles uncertainty in the stated spectral type and adds an additional half-type uncertainty to main sequence stars. WR star unceratainty can be handled too, but that gets a little more case-specific.

In [ ]:
# Mass loss rate (median and uncertainty)
mdot_median, mdot_unc = catr.get_mass_loss_rate()
# Momentum flux
mvflux_median, mvflux_unc = catr.get_momentum_flux()
# Mechanical luminosity from winds
ke_median, ke_unc = catr.get_mechanical_luminosity()
# FUV luminosity (I called this flux in the function, might have to go in and fix that)
fuv_median, fuv_unc = catr.get_FUV_flux()
# Ionizing luminosity
ionizing_median, ionizing_unc = catr.get_ionizing_flux()
# Total cluster mass
mass_median, mass_unc = catr.get_stellar_mass()
# Bolometric luminosity
lum_median, lum_unc = catr.get_bolometric_luminosity()

In [ ]:
# Helper function to print these values
def print_val_err(val: u.Quantity, err: tuple[u.Quantity, u.Quantity], exp=True, extra_f=None):
    """
    :param val: the median value
    :param err: tuple(low value, high value) representing uncertainty bounds
        The errors I am working with are stated as "-low, +high"
        If you subtract low value, you get the 16% value
        If you add the high value, you get the 84% value
        The median is the 50% value
    :param exp: show the numbers in exponential notation (10000 -> 1E+4)
    :param extra_f: some callable to be applied to every value before it is printed
        This could be anything that will still return a Quantity.
    """
    if extra_f is None:
        extra_f = lambda x : x # identity function
    val = f"{extra_f(val):.2E}" if exp else f"{extra_f(val):.2f}"
    str_func = lambda x : f"{extra_f(x).to_value():+.1E}" if exp else f"{extra_f(x).to_value():+.1f}"
    lo, hi = (str_func(x) for x in err)
    return f"{val} [{lo}, {hi}]"

In [ ]:
# Print all values and make some combo values
print(f"MASS LOSS: {print_val_err(mdot_median, mdot_unc)}")
print(f"MV FLUX:  {print_val_err(mvflux_median, mvflux_unc)}")
print(f"MECH LUM:  {print_val_err(ke_median, ke_unc, extra_f=lambda x: x.to(u.erg/u.s))}")
print(f"FUV LUM:   {print_val_err(fuv_median, fuv_unc)}") # extra_f=lambda x: x.to(u.erg/u.s)
print(f"IONIZING PHOTON FLUX: {print_val_err(ionizing_median, ionizing_unc)}") # units should be 1/time
print(f"STELLAR MASS: {print_val_err(mass_median, mass_unc)}")
print(f"LUMINOSITY:   {print_val_err(lum_median, lum_unc)}")
print(f"MECH/FUV LUM ratio: {(ke_median/fuv_median).decompose():.1E}; MECH/total LUM ratio: {(ke_median/lum_median).decompose():.1E}")

You could also get values for each star to check which ones dominate each property.

In [ ]:
# Get an array of mass values -- good proxy for how well we connected spectral types to models
mass_array = catr.get_array_stellar_mass()
# This is returned as a list (not array, whoops) with length equal to the number of stars.
# Each element is a tuple[median, [low, high]]
for (med, (lo, hi)), star in zip(mass_array, catr):
    print(f"{str(star):.<30}{med.to_value():.>6.1f} {str(med.unit)} ({lo.to_value():.1f}, {hi.to_value():.1f})")

# Part 2: G<sub>0</sub> Map
We can try a more complex product: the G<sub>0</sub> map.
The code to do a lot of this work isn't actually in `scoby` but probably should be.

In [ ]:
# First, make a WCS grid for this map
# I am using my custom "center coordinate"
center_coord = SkyCoord("10 23 58.1 -57 45 49", unit=(u.hourangle, u.deg), frame='fk5') # Specific to this cluster
# Make a 100-pixel/side grid with pixel size such that the image is a few arcmin on a side
side_length_pix = 100  # pixels
side_length_angle = 8*u.arcmin
pixel_scale = (side_length_angle / side_length_pix).to(u.deg).to_value()

from astropy.wcs import WCS
wcs_keywords = {
    'NAXIS': (2, "Number of axes"),
    'NAXIS1': (side_length_pix, "X (j) axis length"), 'NAXIS2': (side_length_pix, "Y (i) axis length"),
    'RADESYS': 'FK5',
    'CTYPE1': ('RA---TAN', "RA projection type"), 'CTYPE2': ('DEC--TAN', "DEC projection type"),

    'CRPIX1': (side_length_pix//2, "[pix] Image reference point"),
    'CRPIX2': (side_length_pix//2, "[pix] Image reference point"),

    'CRVAL1': (center_coord.ra.deg, "[deg] RA of reference point"),
    'CRVAL2': (center_coord.dec.deg, "[deg] DEC of reference point"),

    'CDELT1': -1*pixel_scale, # RA increases to the left, so CDELT1 is negative
    'CDELT2': pixel_scale,

    'PA': (0, "[deg] Position angle of axis 2 (E of N)"),
    'EQUINOX': (2000., "[yr] Equatorial coordinates definition"),
}
image_wcs = WCS(wcs_keywords)

In [ ]:
# Define Habing unit (standard unit for G0 values)
radiation_field_1d = 1.6e-3 * u.erg / (u.cm**2 * u.s)

In [ ]:
import numpy as np

# Line-of-sight distance to this cluster
observer_distance = 4.16*u.kpc

# Make function to create the map of inverse square distance, a useful quantity here
def inv_dist_func(coord):
    # I already have a utility function for a lot of this work
    return 1./(4*np.pi * scoby.utils.distance_from_point_pixelgrid(coord, image_wcs, observer_distance)**2.)

# Apply function to each star, creating a 2D image array of 1 / (4pi r^2) for each star
inv_distance_arrays = u.Quantity(list(df['SkyCoord'].apply(inv_dist_func).values))

In [ ]:
inv_distance_arrays.shape # 37 stars, each has an image of 1/(4pi r^2)

In [ ]:
# Make a function to apply to the cluster's list of integrated flux values
# This will be applied to each realization of the cluster's integrated flux values
# since we bootstrap to get the map's uncertainty
def illumination_distance(flux_array):
    # Given a 1D array of flux values (one for each star), multiply by the 1/(4pi r^2) array
    # This will yield (flux / 4pi r^2) for each star at each pixel
    return inv_distance_arrays * flux_array[:, np.newaxis, np.newaxis]

In [ ]:
# Use CatalogResolver to apply this function to FUV flux values
# CatalogResolver will do a bunch of error estimation as well
value, uncertainty = catr.get_FUV_flux(map_function=illumination_distance)

In [ ]:
# Convert units
def fix_units(x):
    # Convert the solLum/kpc^2 units to Habing units
    return (x / radiation_field_1d).decompose()

value = fix_units(value)
uncertainty = tuple(fix_units(x) for x in uncertainty)

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
plt.subplot(111, projection=image_wcs)
plt.imshow(np.log10(value), origin='lower', vmin=3, vmax=6)
plt.colorbar(label='log$_{10}$ $G_0$ (Habing units)')
plt.title("G$_0$ map of Westerlund 2");

You can also plot the error bars as images. There are some interesting patterns that I have not thought much about or looked into.

The uncertainty on this map is approximately 10%.

In [ ]:
plt.figure(figsize=(16, 5))
plt.subplot(121, projection=image_wcs)
plt.imshow(np.log10(np.abs(uncertainty[0])), vmin=2, vmax=5)
plt.title("Uncertainty lower bound")
plt.subplot(122, projection=image_wcs)
plt.imshow(np.log10(np.abs(uncertainty[0])), vmin=2, vmax=5)
plt.title("Uncertainty upper bound")
plt.colorbar();